<a href="https://colab.research.google.com/github/akr-25/Language-Translation/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# English to German Translation model

## Importing Libraries

In [ ]:
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py

--2022-07-12 06:35:26--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2813 (2.7K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-07-12 06:35:26 (38.4 MB/s) - ‘helper.py’ saved [2813/2813]



In [1]:
import collections

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1226259950617150990
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14465892352
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7471803164431816584
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


# Downloading WMT14.en-de dataset

In [3]:
!mkdir data && cd data && mkdir vocab && mkdir train && mkdir test

In [4]:
!cd data/train && curl -o train.en.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  615M  100  615M    0     0  15.2M      0  0:00:40  0:00:40 --:--:-- 17.2M


In [5]:
!cd data/train && curl -o train.de.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  684M  100  684M    0     0  15.5M      0  0:00:44  0:00:44 --:--:-- 17.1M


In [6]:
with open("./data/train/train.de.txt") as myfile:
    headD = [next(myfile) for x in range(5)]

with open("./data/train/train.en.txt") as engFile:
    headE = [next(engFile) for x in range(5)]

for idx in range(5):
  print(headD[idx])
  print(headE[idx])

iron cement ist eine gebrauchs ##AT##-##AT## fertige Paste , die mit einem Spachtel oder den Fingern als Hohlkehle in die Formecken ( Winkel ) der Stahlguss -Kokille aufgetragen wird .

iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .

Nach der Aushärtung schützt iron cement die Kokille gegen den heissen , abrasiven Stahlguss .

iron cement protects the ingot against the hot , abrasive steel casting process .

feuerfester Reparaturkitt für Feuerungsanlagen , Öfen , offene Feuerstellen etc.

a fire restant repair cement for fire places , ovens , open fireplaces etc .

Der Bau und die Reparatur der Autostraßen ...

Construction and repair of highways and ...

die Mitteilungen sollen den geschäftlichen kommerziellen Charakter tragen .

An announcement must be commercial character .



In [7]:
#  e.g., "rich-text format" --> rich ##AT##-##AT## text format.

In [8]:
!cd data/test && curl -o newstest2012.en.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.en
!cd data/test && curl -o newstest2012.de.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.de
!cd data/test && curl -o newstest2013.en.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2013.en
!cd data/test && curl -o newstest2013.de.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2013.de
!cd data/test && curl -o newstest2014.en.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2014.en
!cd data/test && curl -o newstest2014.de.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2014.de
!cd data/test && curl -o newstest2015.en.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2015.en
!cd data/test && curl -o newstest2015.de.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2015.de

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  396k  100  396k    0     0   187k      0  0:00:02  0:00:02 --:--:--  187k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  459k  100  459k    0     0   205k      0  0:00:02  0:00:02 --:--:--  205k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  346k  100  346k    0     0   168k      0  0:00:02  0:00:02 --:--:--  168k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  395k  100  395k    0     0   174k      0  0:00:02  0:00:02 --:--:--  174k
  % Total    % Received % Xferd  Average Speed   Tim

In [9]:
!cd data/vocab && curl -o vocab.50K.en.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en
!cd data/vocab && curl -o vocab.50K.de.txt https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.de

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  394k  100  394k    0     0   191k      0  0:00:02  0:00:02 --:--:--  191k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  493k  100  493k    0     0   220k      0  0:00:02  0:00:02 --:--:--  220k


In [10]:
with open("./data/vocab/vocab.50K.de.txt") as myfile:
    headD = [next(myfile) for x in range(5)]

with open("./data/vocab/vocab.50K.en.txt") as engFile:
    headE = [next(engFile) for x in range(5)]

for idx in range(5):
  print(headD[idx])
  print(headE[idx])

<unk>

<unk>

<s>

<s>

</s>

</s>

,

the

.

,



In [11]:
english_sentences = []
german_sentences = []

with open('./data/train/train.en.txt') as eng:
  english_sentences = [next(eng) for x in range(1000)]

with open('./data/train/train.de.txt') as de:
  german_sentences = [next(de) for x in range(1000)]

In [12]:
print(len(english_sentences))
print(len(german_sentences))

1000
1000


In [13]:
for i in range(5):
  print(english_sentences[i])
  print(german_sentences[i])

iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .

iron cement ist eine gebrauchs ##AT##-##AT## fertige Paste , die mit einem Spachtel oder den Fingern als Hohlkehle in die Formecken ( Winkel ) der Stahlguss -Kokille aufgetragen wird .

iron cement protects the ingot against the hot , abrasive steel casting process .

Nach der Aushärtung schützt iron cement die Kokille gegen den heissen , abrasiven Stahlguss .

a fire restant repair cement for fire places , ovens , open fireplaces etc .

feuerfester Reparaturkitt für Feuerungsanlagen , Öfen , offene Feuerstellen etc.

Construction and repair of highways and ...

Der Bau und die Reparatur der Autostraßen ...

An announcement must be commercial character .

die Mitteilungen sollen den geschäftlichen kommerziellen Charakter tragen .



In [14]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
german_words_counter = collections.Counter([word for sentence in german_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} german words.'.format(len([word for sentence in german_sentences for word in sentence.split()])))
print('{} unique german words.'.format(len(german_words_counter)))
print('10 Most common words in the german dataset:')
print('"' + '" "'.join(list(zip(*german_words_counter.most_common(10)))[0]) + '"')

26089 English words.
4695 unique English words.
10 Most common words in the English dataset:
"," "the" "." "of" "and" "a" "to" "in" "is" "for"

24903 german words.
5895 unique german words.
10 Most common words in the german dataset:
"," "." "und" "der" "die" "in" "##AT##-##AT##" "ist" "den" "von"


In [15]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [16]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [17]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_german_sentences, english_tokenizer, german_tokenizer =\
    preprocess(english_sentences, german_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_german_sequence_length = preproc_german_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
german_vocab_size = len(german_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max german sentence length:", max_german_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("german vocabulary size:", german_vocab_size)

Data Preprocessed
Max English sentence length: 88
Max german sentence length: 91
English vocabulary size: 4180
german vocabulary size: 5439


In [18]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [19]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, german_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param german_vocab_size: Number of unique german words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(german_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_german_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_german_sentences.shape[-2], 1))

In [20]:
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_german_sequence_length,
    english_vocab_size,
    german_vocab_size)

print(simple_rnn_model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 91, 256)           198912    
                                                                 
 time_distributed (TimeDistr  (None, 91, 1024)         263168    
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 91, 1024)          0         
                                                                 
 time_distributed_1 (TimeDis  (None, 91, 5439)         5574975   
 tributed)                                                       
                                                                 
Total params: 6,037,055
Trainable params: 6,037,055
Non-trainable params: 0
_________________________________________________________________
None


In [21]:

simple_rnn_model.fit(tmp_x, preproc_german_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 8.6013 - accuracy: 0.3983 - val_loss: nan - val_accuracy: 0.6581
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 8.0222 - accuracy: 0.7815 - val_loss: nan - val_accuracy: 0.6581
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 7.0728 - accuracy: 0.7815 - val_loss: nan - val_accuracy: 0.6543
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 2.4841 - accuracy: 0.7611 - val_loss: nan - val_accuracy: 0.6581
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 1.9821 - accuracy: 0.7814 - val_loss: nan - val_accuracy: 0.6581
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 1.9451 - accuracy: 0.7815 - val_loss: nan - val_accuracy: 0.6585
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 1.9497 - accuracy: 0.7804 - val_loss: nan - val_accuracy: 0.6588
Epoch 8/10
1/1 [==============================] - 1s 1s/step -

In [35]:

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], german_tokenizer))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [36]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, german_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param german_vocab_size: Number of unique german words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(german_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_german_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_german_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_german_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(german_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_german_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], german_tokenizer))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 91, 256)           1070336   
                                                                 
 gru_1 (GRU)                 (None, 91, 256)           394752    
                                                                 
 time_distributed_2 (TimeDis  (None, 91, 1024)         263168    
 tributed)                                                       
                                                                 
 dropout_1 (Dropout)         (None, 91, 1024)          0         
                                                                 
 time_distributed_3 (TimeDis  (None, 91, 5440)         5576000   
 tributed)                                                       
                                                                 
Total params: 7,304,256
Trainable params: 7,304,256
No

In [38]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], german_tokenizer))

print("\nCorrect Translation:")
print(german_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
at at at at at at at at at at at at at at at at at at at at at at at at at at at <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
['iron cement ist eine gebrauchs ##AT##-##AT## fertige Paste , die mit einem Spachtel oder den Fingern als Hohlkehle in die Formecken ( Winkel ) der Stahlguss -Kokille aufgetragen wird .\n']

Original text:
['iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .\n']


In [39]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, german_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param german_vocab_size: Number of unique german words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(german_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_german_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_german_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_german_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(german_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_german_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], german_tokenizer))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 91, 256)           1070336   
                                                                 
 gru_2 (GRU)                 (None, 91, 256)           394752    
                                                                 
 time_distributed_4 (TimeDis  (None, 91, 1024)         263168    
 tributed)                                                       
                                                                 
 dropout_2 (Dropout)         (None, 91, 1024)          0         
                                                                 
 time_distributed_5 (TimeDis  (None, 91, 5440)         5576000   
 tributed)                                                       
                                                                 
Total params: 7,304,256
Trainable params: 7,304,256
No

In [41]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, german_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param german_vocab_size: Number of unique german words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    # Hyperparameters
    learning_rate = 0.001
    
    # Build the layers    
    model = Sequential()
    # Encoder
    model.add(GRU(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(german_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_german_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_german_sentences.shape[-2], 1))

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_german_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(german_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_german_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 256)               198912    
                                                                 
 repeat_vector (RepeatVector  (None, 91, 256)          0         
 )                                                               
                                                                 
 gru_4 (GRU)                 (None, 91, 256)           394752    
                                                                 
 time_distributed_6 (TimeDis  (None, 91, 1024)         263168    
 tributed)                                                       
                                                                 
 dropout_3 (Dropout)         (None, 91, 1024)          0         
                                                                 
 time_distributed_7 (TimeDis  (None, 91, 5440)        

In [43]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], german_tokenizer))

print("\nCorrect Translation:")
print(german_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
['iron cement ist eine gebrauchs ##AT##-##AT## fertige Paste , die mit einem Spachtel oder den Fingern als Hohlkehle in die Formecken ( Winkel ) der Stahlguss -Kokille aufgetragen wird .\n']

Original text:
['iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .\n']
